In [2]:
import tensorflow as tf
import numpy as np

# tf.logging.set_verbosity(tf.logging.INFO)

In [16]:
# Define Model

def cnn_fn(features, labels, mode):
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])  # You also can use 1 x 784 vector

    # Convolutional Layer #1
    conv1 = tf.keras.layers.Conv2D(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu
    )

    # Pooling Layer #1
    pool1 = tf.keras.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.keras.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
    )
    pool2 = tf.keras.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Convolutional Layer #3 and Pooling Layer #3
    conv3 = tf.keras.layers.conv2d(
        inputs=pool2,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
    )
    pool3 = tf.keras.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=1)

    # Convolutional Layer #4 and Pooling Layer #4
    conv4 = tf.keras.layers.conv2d(
        inputs=pool3,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
    )
    pool4 = tf.keras.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=1)

    # Dense Layer
    pool4_flat = tf.reshape(pool4, [-1, 5 * 5 * 64])
    dense = tf.keraslayers.dense(inputs=pool4_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.keras.layers.dropout(
        inputs=dense, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Output logits Layer
    logits = tf.keras.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    # In predictions, return the prediction value, do not modify
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Select your loss and optimizer from tensorflow API
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)  # Refer to tf.losses

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)  # Refer to tf.train
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [17]:
# Write your dataset path
#dataset_train = np.load('/content/gdrive/My Drive/mnist-cnn-test/train.npy')
#dataset_eval = np.load('/content/gdrive/My Drive/mnist-cnn-test/valid.npy')
#test_data = np.load('/content/gdrive/My Drive/mnist-cnn-test/test.npy')

#dataset_train = np.load('./train.npy')
#dataset_eval = np.load('./valid.npy')
#test_data = np.load('./test.npy')
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'
path = tf.keras.utils.get_file('mnist.npz', DATA_URL)

with np.load(path) as data:
    train_data = data['x_train']#dataset_train[:, :784]
    train_labels = data['y_train']#dataset_train[:, 784].astype(np.int32)
    eval_data = data['x_test']#[:, :784]
    eval_labels = data['y_test']#dataset_eval[:, 784].astype(np.int32)

# Save model and checkpoint
classifier = tf.estimator.Estimator(model_fn=cnn_fn, model_dir="./model")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.estimator.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
#logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

# Train the model. You can train your model with specific batch size and epoches
train_input = tf.compat.v1.estimator.inputs.numpy_input_fn(x={"x": train_data},
                                                    y=train_labels, batch_size=100, num_epochs=None, shuffle=True)
#train_input = tf.estimator.inputs.numpy_input_fn(x={"x": train_data},
#                                                    y=train_labels, batch_size=100, num_epochs=None, shuffle=True)
classifier.train(input_fn=train_input, steps=20000, hooks=[logging_hook])

# Eval the model. You can evaluate your trained model with validation data
eval_input = tf.estimator.inputs.numpy_input_fn(x={"x": eval_data},
                                                y=eval_labels, num_epochs=1, shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input)

## ----------- Do not modify!!! ------------ ##
# Predict the test dataset
pred_input = tf.estimator.inputs.numpy_input_fn(x={"x": test_data}, shuffle=False)
pred_results = classifier.predict(input_fn=pred_input)
result = np.asarray([x.values()[1] for x in list(pred_results)])
## ----------------------------------------- ##

# np.save('./result.npy', result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


AttributeError: module 'keras.api._v2.keras.layers' has no attribute 'Convolution2d'